In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

import explore

In [2]:
df = explore.make_initial_df('repo_source')
df = explore.add_new_columns(df)

In [3]:
df.head()

,index,repo,language,readme_contents,basic_clean,clean_tokes,lemmatized,clean_lemmatized,gen_language,without_numbers,num_words,num_unique_words,link_counts,py_extensions,js_extensions,ipynb_extensions
0,1,tokyo-metropolitan-gov/covid19,Vue,# 東京都 新型コロナウイルス感染症対策サイト\n\n![](https://github....,\n\nhttpsgithubcomtokyometropolitangovcovid1...,[httpsgithubcomtokyometropolitangovcovid19work...,httpsgithubcomtokyometropolitangovcovid19workf...,httpsgithubcomtokyometropolitangovcovid19workf...,other,httpsgithubcomtokyometropolitangovcovid19workf...,19,19,3,0,0,0
1,3,covid19india/covid19india-react,JavaScript,"<p align=""center"">\n<img src=""https://lh3.goog...",\n\n\n\n\n heres our data api\n \n\n setup\n\...,"[heres, our, data, api, setup, npm, i, npm, st...",here our data api setup npm i npm start mainta...,data api setup npm npm start maintainer jeremy...,JavaScript,data api setup npm npm start maintainer jeremy...,50,47,7,0,0,0
2,5,ieee8023/covid-chestxray-dataset,Jupyter Notebook,\n#### 🛑 Note: please do not claim diagnostic ...,\n note please do not claim diagnostic perfor...,"[note, please, do, not, claim, diagnostic, per...",note please do not claim diagnostic performanc...,note please claim diagnostic performance model...,Jupyter Notebook,note please claim diagnostic performance model...,663,384,31,0,0,0
3,7,ahmadawais/corona-cli,JavaScript,"<h4 align=""center"">\n <a href=""https://gith...",\n \n \n \n \n \n\ntrack th...,"[track, the, coronavirus, disease, covid19, or...",track the coronavirus disease covid19 or the n...,track coronavirus disease covid19 novel corona...,JavaScript,track coronavirus disease covid19 novel corona...,823,413,70,0,2,0
4,8,ExpDev07/coronavirus-tracker-api,Python,"<h1 align=""center"">\n Coronavirus Tracker A...",\n coronavirus tracker api\n\n\nprovides up...,"[coronavirus, tracker, api, provides, uptodate...",coronavirus tracker api provides uptodate data...,coronavirus tracker api provides uptodate data...,Python,coronavirus tracker api provides uptodate data...,1032,480,144,2,1,0


In [4]:
vectorized_df = explore.make_vectorized_df(df)

In [5]:
vectorized_df.head()

,00,002,01,02,04162020,05,05132020,05262020,06012020,0no,...,zinc,zingming,zoom,zum,num_words,num_unique_words,link_counts,py_extensions,js_extensions,ipynb_extensions
0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,19,19,3,0,0,0
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,50,47,7,0,0,0
2,0.630423,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.022515,...,0.0,0.0,0.0,0.0,663,384,31,0,0,0
3,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,823,413,70,0,2,0
4,0.000000,0.0,0.014731,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,1032,480,144,2,1,0


In [6]:
X = vectorized_df
y = df.gen_language

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.2)

train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

lm = LogisticRegression().fit(X_train, y_train)

train['predicted'] = lm.predict(X_train)
test['predicted'] = lm.predict(X_test)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [7]:
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.predicted, train.actual))
print('---')
print(classification_report(train.actual, train.predicted))

Accuracy: 80.00%
---
Confusion Matrix
actual            JavaScript  Jupyter Notebook  Python  other
predicted                                                    
JavaScript                 3                 0       0      0
Jupyter Notebook           0                 3       0      0
Python                     1                 0       4      0
other                      1                 1       1      6
---
                  precision    recall  f1-score   support

      JavaScript       1.00      0.60      0.75         5
Jupyter Notebook       1.00      0.75      0.86         4
          Python       0.80      0.80      0.80         5
           other       0.67      1.00      0.80         6

        accuracy                           0.80        20
       macro avg       0.87      0.79      0.80        20
    weighted avg       0.85      0.80      0.80        20

